In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 2
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
Progress: 100% - Completed
讀取3秒資料


D:\Hu_code\ResNet18_ESC10\ESC10_kFold_5s\..\TrainDataPreprocessing.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 29.66 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.0032
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_5S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 1.6466869115829468 Train_acc: 20.625000%
Train Epoch: 2/90 Train_Loss: 1.2303014993667603 Train_acc: 45.625000%
Train Epoch: 3/90 Train_Loss: 1.0395668745040894 Train_acc: 56.250000%
Train Epoch: 4/90 Train_Loss: 0.8941404223442078 Train_acc: 61.250000%
Train Epoch: 5/90 Train_Loss: 0.9876208901405334 Train_acc: 68.437500%
Train Epoch: 6/90 Train_Loss: 0.8095186948776245 Train_acc: 70.312500%
Train Epoch: 7/90 Train_Loss: 0.7689327001571655 Train_acc: 78.125000%
Train Epoch: 8/90 Train_Loss: 0.598947286605835 Train_acc: 77.812500%
Train Epoch: 9/90 Train_Loss: 0.4299106299877167 Train_acc: 86.250000%
Train Epoch: 10/90 Train_Loss: 0.5017834305763245 Train_acc: 82.500000%
Train Epoch: 11/90 Train_Loss: 0.70131957

Train Epoch: 15/90 Train_Loss: 0.26421821117401123 Train_acc: 92.187500%
Train Epoch: 16/90 Train_Loss: 0.2026035338640213 Train_acc: 93.125000%
Train Epoch: 17/90 Train_Loss: 0.14109843969345093 Train_acc: 92.187500%
Train Epoch: 18/90 Train_Loss: 0.269559383392334 Train_acc: 92.187500%
Train Epoch: 19/90 Train_Loss: 0.17263653874397278 Train_acc: 94.375000%
Train Epoch: 20/90 Train_Loss: 0.11754315346479416 Train_acc: 95.625000%
Train Epoch: 21/90 Train_Loss: 0.1718256175518036 Train_acc: 93.125000%
Train Epoch: 22/90 Train_Loss: 0.2551898658275604 Train_acc: 94.375000%
Train Epoch: 23/90 Train_Loss: 0.08134159445762634 Train_acc: 94.687500%
Train Epoch: 24/90 Train_Loss: 0.19089917838573456 Train_acc: 94.375000%
Train Epoch: 25/90 Train_Loss: 0.1412549614906311 Train_acc: 96.875000%
Train Epoch: 26/90 Train_Loss: 0.08337707072496414 Train_acc: 93.750000%
Train Epoch: 27/90 Train_Loss: 0.02862217091023922 Train_acc: 97.500000%
Train Epoch: 28/90 Train_Loss: 0.13027863204479218 Train_

Train Epoch: 32/90 Train_Loss: 0.029715508222579956 Train_acc: 99.687500%
Train Epoch: 33/90 Train_Loss: 0.053119875490665436 Train_acc: 98.125000%
Train Epoch: 34/90 Train_Loss: 0.13975857198238373 Train_acc: 98.750000%
Train Epoch: 35/90 Train_Loss: 0.02148505486547947 Train_acc: 99.687500%
Train Epoch: 36/90 Train_Loss: 0.10261832177639008 Train_acc: 98.750000%
Train Epoch: 37/90 Train_Loss: 0.03581014275550842 Train_acc: 98.750000%
Train Epoch: 38/90 Train_Loss: 0.04652142524719238 Train_acc: 99.687500%
Train Epoch: 39/90 Train_Loss: 0.02648296020925045 Train_acc: 99.375000%
Train Epoch: 40/90 Train_Loss: 0.037609636783599854 Train_acc: 99.687500%
Train Epoch: 41/90 Train_Loss: 0.014328444376587868 Train_acc: 100.000000%
Train Epoch: 42/90 Train_Loss: 0.01703011803328991 Train_acc: 99.062500%
Train Epoch: 43/90 Train_Loss: 0.01747756637632847 Train_acc: 99.687500%
Train Epoch: 44/90 Train_Loss: 0.01262957789003849 Train_acc: 99.687500%
Train Epoch: 45/90 Train_Loss: 0.0183121878653

Train Epoch: 49/90 Train_Loss: 0.01565893180668354 Train_acc: 100.000000%
Train Epoch: 50/90 Train_Loss: 0.01031288132071495 Train_acc: 99.687500%
Train Epoch: 51/90 Train_Loss: 0.0033268583938479424 Train_acc: 100.000000%
Train Epoch: 52/90 Train_Loss: 0.023193927481770515 Train_acc: 98.125000%
Train Epoch: 53/90 Train_Loss: 0.008543580770492554 Train_acc: 99.687500%
Train Epoch: 54/90 Train_Loss: 0.006960290018469095 Train_acc: 99.687500%
Train Epoch: 55/90 Train_Loss: 0.013155153021216393 Train_acc: 99.687500%
Train Epoch: 56/90 Train_Loss: 0.014045478776097298 Train_acc: 100.000000%
Train Epoch: 57/90 Train_Loss: 0.012644343078136444 Train_acc: 100.000000%
Train Epoch: 58/90 Train_Loss: 0.01069845911115408 Train_acc: 100.000000%
Train Epoch: 59/90 Train_Loss: 0.06807363778352737 Train_acc: 99.687500%
Train Epoch: 60/90 Train_Loss: 0.00807160884141922 Train_acc: 99.687500%
Train Epoch: 61/90 Train_Loss: 0.015202753245830536 Train_acc: 99.687500%
Train Epoch: 62/90 Train_Loss: 0.0042

Train Epoch: 65/90 Train_Loss: 0.002867788542062044 Train_acc: 100.000000%
Train Epoch: 66/90 Train_Loss: 0.0023649632930755615 Train_acc: 100.000000%
Train Epoch: 67/90 Train_Loss: 0.006307380739599466 Train_acc: 100.000000%
Train Epoch: 68/90 Train_Loss: 0.004700212273746729 Train_acc: 100.000000%
Train Epoch: 69/90 Train_Loss: 0.004503630567342043 Train_acc: 100.000000%
Train Epoch: 70/90 Train_Loss: 0.003749116323888302 Train_acc: 100.000000%
Train Epoch: 71/90 Train_Loss: 0.0031460931058973074 Train_acc: 100.000000%
Train Epoch: 72/90 Train_Loss: 0.004864520393311977 Train_acc: 100.000000%
Train Epoch: 73/90 Train_Loss: 0.005925794597715139 Train_acc: 100.000000%
Train Epoch: 74/90 Train_Loss: 0.004120021127164364 Train_acc: 100.000000%
Train Epoch: 75/90 Train_Loss: 0.018433958292007446 Train_acc: 100.000000%
Train Epoch: 76/90 Train_Loss: 0.016499100252985954 Train_acc: 99.375000%
Train Epoch: 77/90 Train_Loss: 0.0019000264583155513 Train_acc: 100.000000%
Train Epoch: 78/90 Trai

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()